In [3]:
"""
Import + test/train sets to NumPy Arrays
"""
# Import libraries & data - Set up training/test set ndarrays
import pandas as pd
import numpy as np

# String of path to training and test csv's 
train_data_location = "/Users/FATMac/Downloads/titanic_train.csv"
test_data_location = "/Users/FATMac/Downloads/titanic_test.csv"

df_train = pd.read_csv(train_data_location)
df_test = pd.read_csv(test_data_location)

train_set = df_train.values
test_set = df_test.values

In [4]:
"""
Data Wrangling
Data should be reshaped as m x n design matrix.
Examples are row vectors (qty m)
Features are column vectors (qty N = n - 1)
"""
# Extract single feature examples ("Pclass") 
train = train_set[:,[2, 5, 1]].astype(float)
test = test_set[:,[1, 4]].astype(float)

#if(train.ndim == 1):
#    train = train[:, np.newaxis]

# Mask out examples (rows) w/ NaN's in training set
train = train[~np.isnan(train).any(axis=1)]
test = test[~np.isnan(test).any(axis=1)]

# Prepend constant offset term to X for model intercept
train = np.insert(train, 0, 1, axis=1)
test = np.insert(test, 0, 1, axis=1)

X = train[:, 0:3] # slice feature values (first n - 1 columns)
y = train[:, -1] # slice out labels (last column)
X_test = test

# Feature scaling - Every column gets scaled
N = X.shape[1]
for i in range(N):
    X[:, i] = (X[:, i] - np.mean(X[:, i])) / np.abs(np.max(X[:, i]))

In [5]:
"""
- Model definitions -
In array shape comments, m is num examples; n is num features + 1
"""
# A Numerically Stable Sigmoid
# "Z": ndarray shape (m, )
sigmoid = lambda Z : np.where(Z > 0., 
                              1. / (1. + np.exp(-Z)), 
                              np.exp(Z) / (1. + np.exp(Z)))

# Hypothesis: Decision boundary is a linear combination of feature
# values w/ learnable intercept, squished by sigmoid...
# "theta": ndarray shape (n, )
# "X": ndarray shape (m, n)
def h(theta, X):
    return sigmoid(X.dot(theta))

# Objective function to Optimize (not called): 
#     log-likelihood is convex ensuring convergence
#     L2 Regularized (avoid overfitting w/ large weights)
#
# "X": input feature values ndarray shape (m, n)
# "Y": observed labels from training data ndarray shape (m, )
# "theta": model parameter ndarray shape (m, n)
def J(X, Y, theta, tune):
    # p = degree of belief that our model assigns to positive class
    p = h(theta, X)
    # Unpack size of batch
    m, = Y.shape
    
    # Regularizer term
    L2_reg = (tune/(2.*m)) * np.dot(theta, theta)
    # Standard logistic regression cost function
    log_likelihood = np.sum(np.multiply(Y, np.log(p)) + (1. - Y) * np.log(1. - p)) / m
    
    # return cost over mini-batch        
    return (-log_likelihood + L2_reg)

def SGD_step(X_mini, y_mini, theta, alpha, tune):
    # Unpack size of mini batch
    m, = y_mini.shape
    
    # Calculate gradient using mini-batch
    err = h(theta, X_mini) - y_mini
    grad_J = (np.dot(X_mini.T, err) + tune * theta) / m
    
    # Compute theta_i+1
    theta_next = theta - alpha * grad_J
    theta_next[0] = theta_next[0] - alpha * np.mean(err)
    
    # return theta_i+1 and new cost (for convergence monitoring)
    return (theta_next, J(X_mini, y_mini, theta, tune))

In [6]:
"""
sklearn baseline for benchmark
"""
# Check for class imbalance
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(fit_intercept=True, verbose=10)
clf.fit(X[:, 1:3], y)

print(clf.intercept_, clf.coef_)
print ('Accuracy from sk-learn: {0}'.format(clf.score(X[:, 1:3], y)))

[LibLinear][-0.42390604] [[-3.13835483 -2.41928263]]
Accuracy from sk-learn: 0.7016806722689075


In [8]:
"""
Model Training and Evaluation
"""
# Hyperparameters of learning algorithm
alpha = 0.1 # learning rate

epochs = 300 # num passes through full training data
iters_per_epoch = 1 # ratio of batch size to mini-batch size

tune = 0. # L2 regularization coefficient for overfitting control

# This yields 50/50 uncertainty in sigmoid on y
theta = np.zeros(shape=X.shape[-1], dtype=np.float64)


# Optimization Loop
for step in range(epochs):
    for iterate in range(iters_per_epoch):
        # mini-batch = full batch (non-stochastic)
        theta, cost = SGD_step(X, y, theta, alpha, tune)
        if step % 10 == 0:
            print("cost ", step, ": ", cost)

print("coefficients: ", theta)
y_hat = np.round(h(theta, X))
print("prediction accuracy: ", 1. - np.sum(np.abs(y_hat - y)/y.shape[0]))

cost  0 :  0.6931471805599453
cost  10 :  0.6907093420560281
cost  20 :  0.6883574260717246
cost  30 :  0.6860880479973973
cost  40 :  0.6838979524722454
cost  50 :  0.6817840096609927
cost  60 :  0.679743211487378
cost  70 :  0.6777726678458273
cost  80 :  0.6758696028102446
cost  90 :  0.674031350856588
cost  100 :  0.6722553531138099
cost  110 :  0.6705391536558082
cost  120 :  0.668880395845282
cost  130 :  0.6672768187387758
cost  140 :  0.6657262535607334
cost  150 :  0.6642266202530787
cost  160 :  0.662775924105637
cost  170 :  0.6613722524716603
cost  180 :  0.6600137715717588
cost  190 :  0.6586987233887017
cost  200 :  0.6574254226548006
cost  210 :  0.6561922539329229
cost  220 :  0.6549976687916136
cost  230 :  0.6538401830742905
cost  240 :  0.6527183742620456
cost  250 :  0.6516308789292033
cost  260 :  0.6505763902904711
cost  270 :  0.6495536558382378
cost  280 :  0.6485614750683516
cost  290 :  0.6475986972925188
coefficients:  [-2.81287135 -1.13753475 -0.26259648]
pr